In [ ]:
!pip insatll wget

In [ ]:
!pip install py4j==0.10.6

In [ ]:
!pip install tensorflow

In [ ]:
!pip install pyspark

In [ ]:
df_data_1.printSchema()

In [ ]:
df_data_1.show()

In [ ]:
df_data_1.select('Primary Type').distinct().show(40)

In [ ]:
df_data_1.count()

In [ ]:
splitted_data = df_data_1.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [ ]:
stringIndexer_label = StringIndexer(inputCol="Primary Type", outputCol="label", handleInvalid="keep").fit(df_data_1)
stringIndexer_des = StringIndexer(inputCol="Description", outputCol="Description_IX", handleInvalid="keep")
stringIndexer_locdes = StringIndexer(inputCol="Location Description", outputCol="Location_Description_IX", handleInvalid="keep")

In [ ]:
vectorAssembler_features = VectorAssembler(inputCols=["Description_IX", "Location_Description_IX", "Year"], outputCol="features")

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

In [ ]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_des, stringIndexer_locdes, vectorAssembler_features, rf, labelConverter])

In [ ]:
train_data.printSchema()

In [ ]:
model_rf = pipeline_rf.fit(train_data)

In [ ]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

In [ ]:
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

In [ ]:
model_artifact = MLRepositoryArtifact(model_rf, training_data = train_data, name="Crime Line prediction")

In [ ]:
saved_model = ml_repository_client.models.save(model_artifact)

In [ ]:
saved_model.meta.available_props()

In [ ]:
print ("modelType: " + saved_model.meta.prop("modelType"))
print ("trainingDataSchema: " + str(saved_model.meta.prop("trainingDataSchema")))
print ("creationTime: " + str(saved_model.meta.prop("creationTime")))
print ("modelVersionHref: " + saved_model.meta.prop("modelVersionHref"))
print ("label: " + saved_model.meta.prop("label"))

In [ ]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

In [ ]:
print (loadedModelArtifact.name)

In [ ]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

In [ ]:
predictions.show(5)

In [ ]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

In [ ]:
!pip install plotly --user

In [ ]:
!pip install cufflinks==0.8.2

In [ ]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go
init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]]))

In [ ]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "Description", "Location Description", "Year").toPandas()

In [ ]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()
crime_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['Description'])]
crime_layout = go.Layout(title='Predicted crime line rate in Chicago from 2001- Present')

fig = go.Figure(data=crime_data, layout=crime_layout)
iplot(fig)